In [ ]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from joblib import Parallel, delayed
from sklearn.svm import LinearSVC
import random
import os
from sklearn.metrics import precision_recall_fscore_support as full_score
from sklearn.model_selection import train_test_split


In [ ]:
def ExistDseparator(TargetNode,Xi, Z, X, alpha):
    flagExist = False
    dsepSet=[]
    #counter=0
    #print_names(Z)
    for i in range(0,(2**len(Z))-1):
        IDsubsetZ_dec = i
        IDsubsetZ_bin = bin(IDsubsetZ_dec)
        subsetZ = getZsubset(IDsubsetZ_bin,Z)
        # no cache
        #print("from exist dseparator")
        dep = Dep(TargetNode,Xi,subsetZ, X, alpha)
        #print(subsetZ)
        #print(dep)
        if (dep==0):
            flagExist = True
            dsepSet = subsetZ
            break
    #print("Module exist d-separator: ",counter)
    return [flagExist,dsepSet]

In [ ]:
def GetCondValues(z, condVars):
    if len(condVars)>1:
        maxz = 1
        for column in condVars:
            maxz*=int(column['name'].split('@')[1]) # equals to prod(sizes)
        d = np.zeros(len(condVars))
        
        div_dim = maxz/int(condVars[0]['name'].split('@')[1])
        num2div = z
        
        for i in range(0,len(condVars)-1):
            d[i] = floor(num2div/div_dim)
            num2div=num2div%div_dim
            div_dim = div_dim/int(condVars[i+1]['name'].split('@')[1])
        d[i] = num2div
    else:
        d = [z]
#     print("Size of condVars was: "+str(len(condVars)))
#     print(d)
#     print(sum([],d))
#     if (len(d)>1):
#         print(d)
#         print(z)
#         print_names(condVars)
    return d

In [ ]:
def Dep(TargetNode, Xi, CondVars, X, alpha):
    #print('in Dep')
    global data_c_size
    
    vvTarget = []
    for i in range(int(TargetNode['name'].split('@')[1])):
        vvTarget.append(i)
    
    vvXi = []
    for i in range(int(Xi['name'].split('@')[1])):
        vvXi.append(i)
    
    szCondVars = 1
    #print(CondVars)
    for column in CondVars:
        #print(column)
        szCondVars *= int(column['name'].split('@')[1])
    
    if data_c_size <= (5 * len(vvTarget)*len(vvXi)*(szCondVars)):
        return 1
    
    if (len(CondVars)==0):
        S = np.zeros((len(vvTarget),len(vvXi)))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(TargetNode['data']) # Target Node
                var2=np.array(Xi['data'])
                # looking up for 0, 1 and 2
                #op1 = tf_mask_var(var1,vvTarget[i])
                op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                #op2 = tf_mask_var(var2,vvXi[j])
                op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                #S[i][j] = tf_dot_mult(op1,op2)
                S[i][j]=np.sum(op1*op2)
                #print(S[i][j])
        G2=0
        N = np.sum(S)
#         print(S)
#         print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
#         print(S[np.where(S>0)])
        Dedf = len((S[np.where(S>0)]))\
                      - len(Si[np.where(Si>0)])\
                      - len(Sj[np.where(Sj>0)]) +1
#         print("Dedf before if: ",Dedf)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        
            
        S = np.zeros((len(vvTarget),len(vvXi),szCondVars))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValues(k,CondVars)
                    #print(condValue)
                    flagDataCondVars = np.ones(len(Xi['data']))
                    for l in range(0,len(CondVars)):
                        X_l = CondVars[l]
                        flagDataCondVars = flagDataCondVars*np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l['data'])))
                        #op1 = tf_mask_var(X_l['data'],condValue[l])
                        #flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                        #flagDataCondVars = flagDataCondVars*op1
                    var1=np.array(TargetNode['data'])
                    var2=np.array(Xi['data'])
                    # looking up for -1, 0 and 1
                    op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    #op1 = tf_mask_var(var1,vvTarget[i])
                    #print(op1)
                    #op2 = tf_mask_var(var2,vvXi[j])
                    op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    #print(op1*op2*flagDataCondVars)
                    S[i][j][k]=np.sum(op1*op2*flagDataCondVars)
        G2 = 0
        
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=0)
        
        #Sjk = sum(S,1);
        #Sik = sum(S,2);
        #Sk = sum(Sjk,2); 
        Dedf = len(S[np.where(S>0)]) -\
                len(Sik[np.where(Sik>0)]) -\
                len(Sjk[np.where(Sjk>0)]) +\
                len(Sk[np.where(Sk>0)])
        if Dedf<1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        #print(S.shape)
        #print(Sjk.shape)
        #print(Sik.shape)
        #print(Sk.shape)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        #print(i,j,k)
                        #test_op =  Sk[k]
                        #test_op = Sik[i][k]
                        #test_op = Sjk[j][k]
                        G2=G2+(S[i][j][k] * np.log(\
                                    (S[i][j][k] * Sk[k])/\
                                    (Sik[i][k] * Sjk[j][k])))
        G2 = 2*G2
    
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
#     print("Target Node: ",TargetNode['name'])
#     print("Node XI: ", Xi['name'])
#     print("\tG2:",G2)
#     print("\tDedf: ",Dedf)
#     print("\tAssoc: ",assoc)
#     print()
#     print()
    return assoc

In [ ]:
def getZsubset(bin_id,Z):
    bin_str=str(bin_id[::-1])
    Zsubset=[]
    for i in range(0,len(bin_str)):
        if bin_str[i]=='1':
            Zsubset.append(Z[i])
    return Zsubset

In [ ]:
def arrayUniverse(TargetNode,X):
    Universe = []
    for key in X:
        #print("Key in X:",key)
        #print("Target Node is: ",TargetNode)
        if (key!=TargetNode):
            append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
            Universe.append(append_dict)
    return Universe

In [ ]:
def arrayX(X):
    returnable=[]
    for key in X:
        #tagged_c = MB_Column(X[key].copy(deep=True).tolist(),key)
        append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
        returnable.append(append_dict)
    return returnable

In [ ]:
def print_names(dict_list):
    print()
    for item in dict_list:
        print(item['name'],end=" ")
    print()

In [ ]:
def MinAssoc(TargetNode, Xi,Z, fixedCondVars, X, alpha):
    
    min_assoc=999
    #counter=0
    #print(fixedCondVars)
    if len(Z)==0:
        #print("from minassoc1")
        #print(fixedCondVars)
        min_assoc = Dep(TargetNode, Xi, fixedCondVars, X, alpha)
        #counter+=1
        subsetZ_min_assoc = fixedCondVars
        #print(min_assoc)
    else:
        #print(2**len(Z)-1)
        for IDsubsetZ_dec in range(0,2**len(Z)-1):
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            #print(len(Xi))
            #print("from minassoc2")
            #print(IDsubsetZ_dec)
            #print(subsetZ)
            subsetZ_assoc=Dep(TargetNode, Xi, fixedCondVars+subsetZ,X,alpha)
            #counter+=1
            #print(subsetZ_assoc[IDsubsetZ_dec])
            if subsetZ_assoc < min_assoc:
                min_assoc = subsetZ_assoc
                subsetZ_min_assoc = fixedCondVars + subsetZ
                if (min_assoc==0):
                    break
    #print("Min Assoc module: ",counter)
    return min_assoc

In [ ]:
def MaxMinHeuristic(TargetNode, CPC, Universe, X, alpha):
    F=[]
    assocF=-1
    Z = CPC
    fixedCondVars = []
    if (len(CPC)>0):
        Z = CPC[0:-1]           # all but the last one   
        fixedCondVars = [CPC[-1]] # we use last one
    
    for i in range(len(Universe)-1,-1,-1):
    #for i in range(0,len(Universe)):
        if (len(Universe[i])==0):
            continue
        minAssoc = MinAssoc(TargetNode,Universe[i], Z,fixedCondVars, X,alpha)
        #print("\nFor "+Universe[i]['name']+"the minAssoc value was: "+str(minAssoc))
        if minAssoc <= 0.0:
            #Universe[i]=[]
            Universe.pop(i)
            continue
        if minAssoc>assocF:
            assocF = minAssoc
            F=Universe[i]
        
        #print(F)
    return [F,assocF,Universe]

In [ ]:
def print_CPC(CPC):
    for i in CPC:
        print(i['name']+" ", end="")

In [ ]:
def MMPC_(TargetNode,Universe,X,alpha):
    
    CPC=[]
    #print(TargetNode)
    # Phase I: Foward
    print("Entering Phase I")
    print("MMPC_beggining: \n"+str(len(Universe)))
    while len(Universe)>0:
        CPC_old = list(CPC) # copy
        maxminheur=MaxMinHeuristic(TargetNode,CPC,list(Universe),X,alpha)
        # maxminheur = [F, assocF, Universe]
        F = maxminheur[0]
        assocF = maxminheur[1]
        #print(assocF)
        Universe = maxminheur[2]
        #print("Universe printing line")
        #print(Universe)
        #print("\n=============================")
        #print(CPC)
        if assocF > 0:
            CPC.append(F)
            indF=Universe.index(F)
            Universe.pop(indF)
        #if (len(CPC)==len(CPC_old)) or (len(CPC)>0.3*(len(Universe)-1)):
        if (len(CPC)==len(CPC_old)):
            break
#         print("Universe actual size:")
#         print(len(Universe))
#         print("CPC actual size:")
#         print(len(CPC))
#         print("CPC contents:")
#         print_CPC(CPC)
    
    # Phase 2: Backward
    print("\nExiting MMPC_")
#     CPC=CPC[::-1]
#     if len(CPC)>1:
#         Z=list(CPC)
#         for i in range(len(CPC)-1,-1,-1):
#             # index is i
#             Z.pop(i)
#             #print("Analyzing D-separator for ",CPC[i]['name'])
#             if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
#                 #print("it did exist! removing from cpc.")
#                 CPC.pop(i)
#     return symmetry_test_v2(TargetNode,CPC,X,alpha)
    return CPC

In [ ]:
def MMPC(TargetNode, X, alpha):
    # The universe will be an array of DataFrame Columns
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [ ]:
def MMPC_Parallel(arg):
    TargetNode, X, alpha = arg
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [ ]:
def getKeysPartition(targetNode, data_keys,n_partitions):
    n_partitions = int(np.log2(len(data_keys)))*2*2  # log2 ( number of variables ) divided by 2
    partitions = []
    partition_diff = int(int(len(data_keys)/n_partitions)/2)
    for i in range(0,n_partitions):
        if (i==0):
            partitions.append(list(data_keys[-partition_diff::])+list(data_keys[0:int(len(data_keys)/n_partitions)]))
            #partitions.append(list(data_keys[0:int(len(data_keys)/n_partitions)]))
            continue
        if (i<n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i:int(len(data_keys)/n_partitions)*(i+1)]))
            #partitions.append(list(data_keys[int(len(data_keys)/n_partitions)*i:int(len(data_keys)/n_partitions)*(i+1)]))
            continue
        if (i==n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i::]))
            #partitions.append(list(data_keys[int(len(data_keys)/n_partitions)*i::]))
            
    for i in range(0,n_partitions):
        for j in range(len(partitions[i])-1,-1,-1):
            if (partitions[i][j]==targetNode):
                partitions[i].pop(j)
        partitions[i] = partitions[i] + [targetNode]
    return partitions,n_partitions

In [ ]:
abc="Hello, I love you, Won't you tell me your name"
abc.split(',')

In [ ]:
%%time
alpha=0.05
print("======================================================================================")
print("Partitioned MMPC_ Algorithm")
print("======================================================================================")
main_directory= '/home/a20114261/alarm_datasets/Alarm10/'
if not os.path.exists(main_directory):
    print("Bad routing.")
PCs=[]
for filename in os.listdir(main_directory):
    print("Analizing "+filename)
    X = pd.read_csv(main_directory+'/'+filename,delimiter='  ',header=None)
    PC=[]
    x_heads = []
    for column in X:
        X[column] = list(map(lambda x: x , X[column]))
    for i in range(0,len(X.keys())):
        max_v=1
        for row in X[X.keys()[i]]:
            if (row>max_v-1):
                max_v+=row-(max_v-1)
        save_i=i+1
        x_heads.append("Node"+str(save_i)+"@"+str(max_v))
    X.columns = x_heads
    data_c_size = len(X[X.keys()[1]])
    data_vars = [0,1,2,3]
    print("======================================================================================")
    
    # selection of target
    for TargetNodeSelected in [x for x in X.keys() if int(x.split('@')[1]) == 2]:
    #  -----------------
        n_partitions = 0
        keys_partitions, n_partitions = getKeysPartition(TargetNodeSelected,X.keys(),n_partitions)
        arg_instances = []
        for i in range(0,n_partitions):
            arg_instances.append((TargetNodeSelected,X[keys_partitions[i]],alpha))
        PC=Parallel(n_jobs=n_partitions)(map(delayed(MMPC_Parallel),arg_instances))
        PC_dict={}
        PC_dict['TargetNode']=TargetNodeSelected
        PC_dict['pcs']=PC
        PC_dict['filename']=filename
        PCs.append(PC_dict)
# for after
n_parallel_forks = n_partitions
    


Exiting MMPC_
16
17
16
Entering Phase I
16
16
MMPC_beggining: 
16
17
Entering Phase I
MMPC_beggining: 
1616
17
16
17

17
17
16
Entering Phase I
MMPC_beggining: 
16
Entering Phase I
Entering Phase I
16
17
17
MMPC_beggining: 
16MMPC_beggining: 
16MMPC_beggining: 
1616
Entering Phase I
16

16
Entering Phase I
Entering Phase I


17
17
MMPC_beggining: 
16MMPC_beggining: 
16
16
Entering Phase I
Entering Phase I
17

16
MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
1616

17
16
16
Entering Phase I
MMPC_beggining: 
1617

16

16
17
Entering Phase I
MMPC_beggining: 
16Entering Phase I
16
17
17
17
Entering Phase I
MMPC_beggining: 
16
16
17
Entering Phase I
17
Entering Phase I
MMPC_beggining: 
16MMPC_beggining: 
1617
MMPC_beggining: 
16



Entering Phase I
16
MMPC_beggining: 
16MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
16

17
Entering Phase I
16
16

16
16
17
Entering Phase I
16
17
17
16
17
Entering Phase I
Entering Phase I
MMPC_beggining: 
1616
MMPC_beggining: 
1617
17
Entering Ph

16Entering Phase I

17
17
17
Entering Phase I

MMPC_beggining: 
16MMPC_beggining: 
1617
MMPC_beggining: 
16
Entering Phase I
17


17
16
MMPC_beggining: 
16
Entering Phase I
16
Entering Phase I
17
16
17
Entering Phase I
MMPC_beggining: 
16MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
16
16

MMPC_beggining: 
1617


15
MMPC_beggining: 
16Entering Phase I
Entering Phase I
16

17
Entering Phase I

MMPC_beggining: 
1616
Entering Phase I
17
MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
16
MMPC_beggining: 
16MMPC_beggining: 
16
Entering Phase I



MMPC_beggining: 
15
34

Exiting MMPC_
Exiting MMPC_


Exiting MMPC_

Exiting MMPC_

Exiting MMPC_
Exiting MMPC_
Exiting MMPC_


Exiting MMPC_
Exiting MMPC_


Exiting MMPC_
Exiting MMPC_



Exiting MMPC_
Exiting MMPC_35

Entering Phase I


Exiting MMPC_
Exiting MMPC_
MMPC_beggining: 
34


Exiting MMPC_
Exiting MMPC_


Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_
Exiting MMPC_


Exiting MMPC_

Exiting MMPC_
Exiting MMPC_


E

Exiting MMPC_35



Exiting MMPC_Entering Phase I


Exiting MMPC_
Exiting MMPC_
MMPC_beggining: 
34


Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_

Exiting MMPC_
16
16
16
17
16
16
17
Entering Phase I
MMPC_beggining: 
16
17
16
Entering Phase I
Entering Phase I
17
Entering Phase I
MMPC_beggining: 
1617
Entering Phase I
MMPC_beggining: 
16MMPC_beggining: 
16
16


17
MMPC_beggining: 
16
17
16
Entering Phase I
17
16
17
Entering Phase I
16
MMPC_beggining: 
1616
16
16
MMPC_beggining: 
16Entering Phase I

17
17
Entering Phase I

17
16
16
17
MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
16Entering Phase I
MMPC_beggining: 
16Entering Phase I
16


17
16
Entering Phase I
MMPC_beggining: 
1617

16
MMPC_beggining: 
16
17
Entering Phase I
17
16
16

Entering Phase I
17
Entering Phase I
MMPC_beggining: 
1616
MMPC_beggini

In [ ]:
initial_universe=[]
for PC_dict in PCs:
    PC_train={}
    PC_train['TargetNode']=PC_dict['TargetNode']
    PC_train['dataset']=PC_dict['filename']
    PC_train['supercpc']=[]
    for pc_subset in PC_dict['pcs']:
        if (len(pc_subset)==0):
            continue
        for e in pc_subset:
            PC_train['supercpc'].append(e['name'].split('@')[0])
    if (len(PC_train['supercpc'])==0):
        continue
    PC_train['supercpc']=list(set(PC_train['supercpc']))
    initial_universe.append(PC_train)

In [ ]:
x_heads_train=[]
for e in x_heads:
    x_heads_train.append(e.split('@')[0])

In [ ]:
#%%capture
import time
PC_mean_score=[]
for filename in os.listdir(main_directory):
    for PC_set in [x for x in initial_universe if x['dataset']==filename]:    
        start = time.time()
        print("For "+PC_set['dataset']+" and Target Node "+PC_set['TargetNode'])
        print("CPC of length "+str(len(PC_set['supercpc'])))
        PC_mean_score.append([filename,CandidateScore(PC_set,PC_set['TargetNode'].split('@')[0],i)])    
        end = time.time()
        #print("Time elapsed: "+str(end - start)+" seconds")
        print()
        print("=================================================================")    


In [ ]:
for e in PC_mean_score:
    print("* Training dataset: "+e[0])
    print("\t- Training dataset: "+e[1]['dataset'])
    print("\t- Target Node: "+e[1]['TargetNode'])
    print("\t- Supercpc of length: "+str(len(e[1]['supercpc'])))
    print("\t- Tests results:")
    for i in e[1]['class_precisions']:
        print("\t\t- ",end=' ')
        print("Test dataset: "+i[1]+"\t G-mean: "+str(g_mean(i[0])))
        #print("\t\t> ",end=' ')
        for j in i[0]:
            print("\t\t   >>>>\t",end='')
            print("Class: "+str(j[0]))
            print("\t\t\t"+"Samples: "+str(j[1]))
            print("\t\t\t"+"Class precision: "+str(j[2]))
        print()

In [ ]:
def CandidateScore(PobDict,TargetEvalNode,intercept_scaling):
    global x_heads_train

    # training and scoring
    X_train_df = pd.read_csv(main_directory+PobDict['dataset'],delimiter='  ',header=None)
    X_train_df.columns=x_heads_train
    
    #X_train, X_test, Y_train, Y_test = train_test_split(X_train_df, X_train_df[TargetEvalNode], test_size=0.1, random_state=0)
    
    X_train = X_train_df
    Y_train = X_train_df[TargetEvalNode]
    
    clf = LinearSVC()
    clf.fit(X_train[PobDict['supercpc']],Y_train)
    
    PobDict['class_precisions']=[]
    for filename in os.listdir(main_directory):
        if (filename == PobDict['dataset']):
            continue
        print("*****")
        print()
        print("Testing on dataset: "+filename)
        X_test = pd.read_csv(main_directory+filename,delimiter='  ',header=None)
        X_test.columns = x_heads_train
        

        # get score values
        precision = []
        values_counter=set(X_train_df[TargetEvalNode])
        for val in values_counter:
            #Y_pred_c = clf.predict(X_train_df.query( TargetEvalNode+'== '+str(i))[PobDict['supercpc']][-100:])
            if (len(X_test.query( TargetEvalNode+'== '+str(val)))==0):
                precision.append([val,0,0])
                continue
            Y_pred_c = clf.predict(X_test.query( TargetEvalNode+'== '+str(val))[PobDict['supercpc']])
            precision.append([val,len(Y_pred_c),Y_pred_c.tolist().count(val)/len(Y_pred_c)])

        for e in range(0,len(precision)):
            print('precision for class '+str(precision[e][0])+' with '+str(precision[e][1])+' samples in dataset : '+str(precision[e][2]))
    
        print("G-mean score: "+str(g_mean(precision)))
        print("*****")
        print()
        #print(clf.score(X_test[PobDict['supercpc'][:10]],Y_test))
        PobDict['class_precisions'].append([precision,filename])
    return PobDict

In [ ]:
def g_mean(precision_arr):
    counter=1
    for e in precision_arr:
        counter=counter*e[2]
    
    return counter**(1/len(precision_arr))